Input - Entrada
 O problema de negócio
 
Selecionar os mais valiosos clientes para formar o programa de fidelidade "INSIDERS"Conjunto de dados com as vendas de produtos durante 1 ano ( Nov 2015 to Dec 2017 )
 Output - Saída
 
A indicação de pessoas para fazer parte do programa de fidelidade "INSIDERS".Relatório com as respostas para as seguintes perguntas:
Quem são as pessoas elegíveis para participar do programa de Insiders ?Quantos clientes farão parte do grupo?Quais as principais características desses clientes ?Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?Qual a expectativa de faturamento desse grupo para os próximos meses ?Quais as condições para uma pessoa ser elegível ao Insiders ?Quais as condições para uma pessoa ser removida do Insiders ?Qual a garantia que o programa Insiders é melhor que o restante da base ?Quais ações o time de marketing pode realizar para aumentar o faturamento?

 Tasks - Processo
 
Quem são as pessoas elegíveis para participar do programa de Insiders ?
O que é ser elegível ? O que é um cliente "valioso" para a empresa ?
Faturamento:
Alto Ticket MédioAlto LTVBaixa Recência ou Alta Frequência ( tempo entre as compras )Alto Basket Size ( quantidade média de produtos comprados )Baixa probabilidade de ChurnPrevisão alta de LTVAlta propensão de compra
Custo:
Baixo número de devoluções
Experiência:  
Média alta de avaliações


Quantos clientes farão parte do grupo?
Número de clientes% em relação ao total de clients
Quais as principais características desses clientes ?
Escrever os principais atributos dos clientes
IdadePaísSalário
Escrever os principais comportamentos de compra dos clients ( métricas de negócio )
Vide acima

Qual a porcentagem de contribuição do faturamento, vinda do Insiders ?
Calcular o faturamento total da empresa durante o ano.Calcular o faturamento (%) apenas do cluster Insiders.
Qual a expectativa de faturamento desse grupo para os próximos meses ?
Cálculo do LTV do grupo InsidersSéries Temporais ( ARMA, ARIMA, HoltWinter, etc )
Quais as condições para uma pessoa ser elegível ao Insiders ?
Qual o período de avaliação ?O "desempenho" do cliente está próximo da média do cluster Insiders.
Quais as condições para uma pessoa ser removida do Insiders ?
O "desempenho" do cliente não está mais próximo da média do cluster Insiders.
Qual a garantia que o programa Insiders é melhor que o restante da base ?
Teste de HipótesesTeste A/B
Quais ações o time de marketing pode realizar para aumentar o faturamento?
DescontosPreferências de escolhaProdutos exclusivos

## Imports

In [1]:
import pandas  as pd
import numpy   as np
import seaborn as sns


from matplotlib import pyplot as plt
from IPython.display import HTML

from pandas_profiling import ProfileReport

from sklearn import cluster as c
from sklearn import metrics as m
from sklearn import preprocessing as pp
from plotly import express as px

from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

## Helper functions

In [2]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    
    plt.style.use('ggplot')
    plt.rcParams['figure.figsize'] = [24,9]
    plt.rcParams['font.size'] = 24
    
    display(HTML('<style>.container { width:100% !important;}</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr', False)
    
    sns.set()

jupyter_settings()

Populating the interactive namespace from numpy and matplotlib


## Load dataset

In [3]:
df_raw = pd.read_csv('../data/raw/data.csv')

In [4]:
df_raw.shape

(541909, 9)

In [5]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Unnamed: 8
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom,NaN
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom,NaN


In [6]:
df_raw.drop('Unnamed: 8',axis = 1, inplace = True)
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


# Data Description

In [7]:
df1 = df_raw.copy()

## Columns Rename

In [8]:
df1.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [9]:
cols_new = ['invoice_no', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price','customer_id', 'country']

In [10]:
df1.columns = cols_new

In [11]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


## Data Dimensions

In [12]:
print(f'Number of Rows:{df1.shape[0]}')
print(f'Number of Columns:{df1.shape[1]}')

Number of Rows:541909
Number of Columns:8


## Data types

In [13]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
customer_id     float64
country          object
dtype: object

In [14]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,17850.0,United Kingdom


In [15]:
# Testing for possible anomalies:

# df1.invoice_no = df1.invoice_no.astype(int)

# invoice_date


## Check NA

In [16]:
df1.isna().sum()

invoice_no           0
stock_code           0
description       1454
quantity             0
invoice_date         0
unit_price           0
customer_id     135080
country              0
dtype: int64

135.080 customers.... 'missing'

## Replace NA

Split dataset into missing vs not_missing:

In [17]:
df_missing = df1.loc[df1['customer_id'].isna(),:]
df_not_missing = df1.loc[~df1['customer_id'].isna(),:]
df_missing.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country
622,536414,22139,NaN,56,29-Nov-16,0.00,NaN,United Kingdom
1443,536544,21773,DECORATIVE ROSE BATHROOM BOTTLE,1,29-Nov-16,2.51,NaN,United Kingdom
1444,536544,21774,DECORATIVE CATS BATHROOM BOTTLE,2,29-Nov-16,2.51,NaN,United Kingdom
1445,536544,21786,POLKADOT RAIN HAT,4,29-Nov-16,0.85,NaN,United Kingdom
1446,536544,21787,RAIN PONCHO RETROSPOT,2,29-Nov-16,1.66,NaN,United Kingdom


In [18]:
df_not_missing.shape

(406829, 8)

In [19]:
df_missing.shape

(135080, 8)

In [20]:
missing_invoice = df_missing['invoice_no'].drop_duplicates().tolist()
missing_invoice[0:10]

['536414',
 '536544',
 '536545',
 '536546',
 '536547',
 '536549',
 '536550',
 '536552',
 '536553',
 '536554']

In [21]:
df_not_missing.loc[df_not_missing['invoice_no'].isin(missing_invoice), :].head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,customer_id,country


In [22]:
# Decision: each NAN customer ID will be a new single customer time purchase, just for training:

# Create Reference:
df_backup = pd.DataFrame(df_missing['invoice_no'].drop_duplicates())
df_backup['customer_id'] = np.arange(19000, 19000+len(df_backup), 1) # linspace or arrange

# merging original w/ reference df:
df1 = pd.merge(df1, df_backup, on = 'invoice_no', how = 'left')
# need to keep columns...
# Command coalesce: it comes from sql, in pandas different:
df1['customer_id'] = df1['customer_id_x'].combine_first(df1['customer_id_y'])
# drop aux columns
df1 = df1.drop(columns=['customer_id_x','customer_id_y'])
df1.isna().sum()

# np.arange (inicial, final, passo)
# np.arange(19000, 19000+len(df_missing), 1) >> até o fim .....


invoice_no         0
stock_code         0
description     1454
quantity           0
invoice_date       0
unit_price         0
country            0
customer_id        0
dtype: int64

In [23]:
# Removing NAs
df1 = df1.dropna(subset =['description','customer_id'])
print('Removed data: {:.2f}'.format(1 - (df1.shape[0] / df_raw.shape[0])))

Removed data: 0.00


In [24]:
df1.shape

(540455, 8)

In [25]:
df1.isna().sum()

invoice_no      0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
country         0
customer_id     0
dtype: int64

## Change dtypes

In [26]:
df1.dtypes

invoice_no       object
stock_code       object
description      object
quantity          int64
invoice_date     object
unit_price      float64
country          object
customer_id     float64
dtype: object

In [27]:
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,29-Nov-16,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,29-Nov-16,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,29-Nov-16,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,29-Nov-16,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,29-Nov-16,3.39,United Kingdom,17850.0


In [28]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format='%d-%b-%y' )
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850.0
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850.0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850.0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850.0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850.0


In [29]:
df1['customer_id'] = df1['customer_id'].astype(int)
df1.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


In [30]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int32
dtype: object

## Descriptive Statistics

In [31]:
#  do the model, then come back

In [32]:
df1.dtypes

invoice_no              object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
country                 object
customer_id              int32
dtype: object

In [33]:
num_attributes = df1.select_dtypes(include =['int64','float64', 'int32'])
cat_attributes = df1.select_dtypes(exclude =['int64','float64', 'int32'])

### Numerical Attributes

In [34]:
# central tendency: mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T
# Dispersion - std dev, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

m = pd.concat([ct1,ct2,d1,d2,d3,d4,d5,d6])
m['index'] = ['Mean','Median', 'Std', 'Min','Max','Range','Skew','Kurtosis']
m.set_index('index', inplace = True)
m.T


index,Mean,Median,Std,Min,Max,Range,Skew,Kurtosis
quantity,9.603129,3.00,218.007397,-80995.00,80995.0,161990.00,-0.281253,120252.449063
unit_price,4.623519,2.08,96.889538,-11062.06,38970.0,50032.06,186.257919,58848.017575
customer_id,16678.153600,16241.00,2907.570262,12346.00,22709.0,10363.00,0.493600,-0.792496


#### Checking on Numerical Attributes  !!!

1. Min for quantity is negative.
2. Unity Price 0.0 ? free item?


### Categorical Attributes

In [35]:
#number of invoices with letters:
number_invs = len(cat_attributes.loc[cat_attributes['invoice_no'].apply(lambda x: bool(re.search( '[^0-9]+', x))), 'invoice_no'].drop_duplicates())
print(f'Invoices with letters: {number_invs}')

#number of stock codes with letters:
number_stk = len(cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search( '[^0-9]+', x))), 'stock_code'].drop_duplicates())
print(f'Stock Codes with letters: {number_stk}')

only_letters_unique_stock_code = cat_attributes.loc[cat_attributes['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x ) ) ), 'stock_code' ].unique()
print(f'Unique stock codes with only letters: {only_letters_unique_stock_code}')

Invoices with letters: 3839
Stock Codes with letters: 1073
Unique stock codes with only letters: ['POST' 'D' 'DOT' 'M' 'S' 'AMAZONFEE' 'm' 'DCGSSBOY' 'DCGSSGIRL' 'PADS'
 'B' 'CRUK']


In [36]:
cat_attributes.sample(1)

,invoice_no,stock_code,description,invoice_date,country
44403,540170,22729,ALARM CLOCK BAKELIKE ORANGE,2017-01-03,United Kingdom


In [37]:
df1.loc[df1['description'] == 'RED RETROSPOT PEG BAG'].sort_values('customer_id').reset_index()

,index,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,355397,567929,21155,RED RETROSPOT PEG BAG,6,2017-09-20,2.55,Portugal,12425
1,420312,572886,21155,RED RETROSPOT PEG BAG,12,2017-10-24,2.55,Spain,12448
2,282167,561627,21155,RED RETROSPOT PEG BAG,6,2017-07-26,2.55,Austria,12453
3,156411,550137,21155,RED RETROSPOT PEG BAG,12,2017-04-12,2.55,Germany,12477
4,309866,C564142,21155,RED RETROSPOT PEG BAG,-1,2017-08-21,2.55,Germany,12477
5,319520,564856,21155,RED RETROSPOT PEG BAG,12,2017-08-29,2.55,Germany,12477
6,383191,570002,21155,RED RETROSPOT PEG BAG,18,2017-10-05,2.55,Germany,12477
7,47998,540458,21155,RED RETROSPOT PEG BAG,6,2017-01-05,2.10,Germany,12501
8,214916,555637,21155,RED RETROSPOT PEG BAG,6,2017-06-04,2.55,France,12535
9,332450,566074,21155,RED RETROSPOT PEG BAG,12,2017-09-07,2.55,France,12535


In [38]:
# df1.loc[df1['invoice_no'] == 'C548995']

In [39]:
df1.sample(1)

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
17979,537789,48138,DOORMAT UNION FLAG,10,2016-12-06,6.75,United Kingdom,14051


#### Invoice No

In [40]:
# There are invoices with letters
df_invoices_w_letters = df1.loc[df1['invoice_no'].apply(lambda x: bool (re.search('[^0-9]+', x))),:]

print('Total number of invoices w/ letters')
print(len(df_invoices_w_letters))

# Are all invoices that start with letters 'negative quantity' one?
print('Total number of invoices w/ letters with negative quantity')
print(len(df_invoices_w_letters[df_invoices_w_letters['quantity'] <0]))

# Yes, all invoices starting with letters are negative quantity.

Total number of invoices w/ letters
9291
Total number of invoices w/ letters with negative quantity
9288


#### Stock Code

In [41]:
df1.loc[df1['stock_code'].apply(lambda x: bool(re.search('^[a-zA-Z]+$', x))),'stock_code'].unique()

# Todo:
# Remove Stock_codes in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']
# New stock_Codes after joining missing data ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',        'DCGSSGIRL', 'PADS', 'B', 'CRUK']
    

array(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
       'DCGSSGIRL', 'PADS', 'B', 'CRUK'], dtype=object)

#### Description

In [42]:
# Todo - delete description

#### Country

In [43]:
df1['country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [44]:
# df1['country'].value_counts(normalize = True).head(4)

In [45]:
# df1[['customer_id', 'country']].drop_duplicates().groupby('country').count().reset_index().sort_values('customer_id', ascending = False).head()

In [46]:
part_1 = %who_ls DataFrame 
part_1

['cat_attributes',
 'ct1',
 'ct2',
 'd1',
 'd2',
 'd3',
 'd4',
 'd5',
 'd6',
 'df1',
 'df_backup',
 'df_invoices_w_letters',
 'df_missing',
 'df_not_missing',
 'df_raw',
 'm',
 'num_attributes']

# Data Filtering

In [47]:
# df3 = df_ref.copy()
df2 = df1.copy()

Makes sense to filter some data: negative quantity values, returns in the table.

2 ways - 
- exclude what has been returned, like customer had never bought the product
- other way to create new feature based on returns. - Return feature would be beneficial to score clients?

In [48]:
df1.loc[df1['unit_price'] > 0.0, ['customer_id', 'description', 'unit_price']].sort_values('unit_price', ascending = True).head(10)
# df1[['customer_id', 'description', 'unit_price']].sort_values('unit_price', ascending = True).head(10))


,customer_id,description,unit_price
157195,13952,PADS TO MATCH ALL CUSHIONS,0.001
279045,15618,PADS TO MATCH ALL CUSHIONS,0.001
361741,13405,Bank Charges,0.001
359871,16198,PADS TO MATCH ALL CUSHIONS,0.001
108088,16422,Discount,0.010
182729,12901,Discount,0.030
355050,16422,Manual,0.030
4287,15838,ROTATING SILVER ANGELS T-LIGHT HLDR,0.030
362028,16629,POPART WOODEN PENCILS ASST,0.040
303023,16488,POPART WOODEN PENCILS ASST,0.040


In [49]:
df2.head()

,invoice_no,stock_code,description,quantity,invoice_date,unit_price,country,customer_id
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2016-11-29,2.55,United Kingdom,17850
1,536365,71053,WHITE METAL LANTERN,6,2016-11-29,3.39,United Kingdom,17850
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2016-11-29,2.75,United Kingdom,17850
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2016-11-29,3.39,United Kingdom,17850
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2016-11-29,3.39,United Kingdom,17850


In [50]:
# Numerical Attributes
# unit price > 0
df2 = df2.loc[df2['unit_price'] >= 0.04, :]  # there are products that price is 0.001

# Categorical Attributes
# stock code != [POST, D, M, DOT, CRUK]  
# New ones ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
#df2 = df2[~df2['stock_code'].isin(['POST' 'D' 'M' 'PADS' 'DOT' 'CRUK'])]  # removing these products...
df2 = df2[~df2['stock_code'].isin(['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK'])]  # removing these products...


# Description - drop it
df2 = df2.drop(columns = 'description', axis = 1)

# Countries
# remove not specific like EU, and unspecified.

df2 = df2[~df2['country'].isin(['European Community', 'Unspecified'])]

# quantity - assuming negative numbers are returns.
df2_returns = df2.loc[df2['quantity'] < 0,:]
df2_purchase = df2.loc[df2['quantity'] > 0,:]


# df2.loc[df2['quantity'] < 0,:].sort_values(['customer_id','description']).head()

# Feature Engineering

In [51]:
# Moving Average 7d,14d,30d...

In [52]:
df3 = df2.copy()

## Feature Creation

In [53]:
# data referencing
df_ref = df2.drop(['invoice_no','stock_code',
                   'quantity','invoice_date','unit_price',
                   'country'], axis = 1).drop_duplicates(ignore_index = True)
df_ref.head()

# Gross Revenue ( qtt * price)
# Recency - latest purchase data for customer
# Frequency - how many products client bought in one year period

,customer_id
0,17850
1,13047
2,12583
3,13748
4,15100


### Gross Revenue

In [54]:
# Gross Revenue:
df2_purchase.loc[:,'gross_revenue'] = df2_purchase.loc[:,'quantity'] * df2_purchase.loc[:,'unit_price']

# # Monetary:
df_monetary = df2_purchase.loc[:,['customer_id', 'gross_revenue']].groupby('customer_id').sum().reset_index()
df_ref = pd.merge(df_ref, df_monetary, how = 'left', on = 'customer_id')
df_ref.isna().sum()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


customer_id       0
gross_revenue    91
dtype: int64

### Recency - Days since last purchase

In [55]:
# Recency - how long has it been since the last puchase.
df_recency = df2_purchase[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index()
df_recency['recency_days'] = (df2_purchase['invoice_date'].max() - df_recency['invoice_date']).dt.days  # using dt, to get int type
df_recency = df_recency[['customer_id','recency_days']].copy()

print('*******Nas before merge:')
print(df_recency.isna().sum())

df_ref = pd.merge(df_ref, df_recency, how = 'left', on = 'customer_id')

print('*******Nas After merge:')
print(df_ref.isna().sum())
# 33 clients never bought anything, onlyu returns... likely data doesn´t comprehend these purchases.

*******Nas before merge:
customer_id     0
recency_days    0
dtype: int64
*******Nas After merge:
customer_id       0
gross_revenue    91
recency_days     91
dtype: int64


### Ammount purchased

In [56]:
# This it just the number of products bought
df_ammt_purchased = df2_purchase[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns ={'invoice_no':'invoice_ammt'})
df_ref = pd.merge(df_ref, df_ammt_purchased, how = 'left', on = 'customer_id')
df_ref.isna().sum()
df_ref.head()

,customer_id,gross_revenue,recency_days,invoice_ammt
0,17850,5391.21,372.0,34.0
1,13047,3232.59,56.0,9.0
2,12583,6705.38,2.0,15.0
3,13748,948.25,95.0,5.0
4,15100,876.00,333.0,3.0


### Ammount products item  purchased

In [57]:
# This it just the number of products bought
df_ammt_purchased = df2_purchase[['customer_id', 'quantity']].groupby('customer_id').sum().rename(columns = {'quantity':'item_ammt'}).reset_index()
df_ref = pd.merge(df_ref, df_ammt_purchased, how = 'left', on = 'customer_id')
df_ref.isna().sum()
df_ref.head()

,customer_id,gross_revenue,recency_days,invoice_ammt,item_ammt
0,17850,5391.21,372.0,34.0,1733.0
1,13047,3232.59,56.0,9.0,1390.0
2,12583,6705.38,2.0,15.0,5028.0
3,13748,948.25,95.0,5.0,439.0
4,15100,876.00,333.0,3.0,80.0


### Ammount different products purchased

In [58]:
# This it just the number of products bought
df_ammt_type_of_prod_purchased = df2_purchase[['customer_id', 'stock_code']].groupby('customer_id').count().rename(columns = {'stock_code':'n_of_dif_products'}).reset_index()
df_ref = pd.merge(df_ref, df_ammt_type_of_prod_purchased, how = 'left', on = 'customer_id')
df_ref.isna().sum()

customer_id           0
gross_revenue        91
recency_days         91
invoice_ammt         91
item_ammt            91
n_of_dif_products    91
dtype: int64

### Average Ticket Value

In [59]:
# Avg Ticket
df_avg_ticket = df2_purchase[['customer_id', 'gross_revenue']].groupby('customer_id').mean().reset_index().rename(columns={'gross_revenue':'avg_ticket'})
df_avg_ticket['avg_ticket'] = np.round(df_avg_ticket['avg_ticket'],2 )
df_ref = pd.merge(df_ref, df_avg_ticket, on = 'customer_id', how = 'left')
df_ref.isna().sum()

# df_ref.head()

customer_id           0
gross_revenue        91
recency_days         91
invoice_ammt         91
item_ammt            91
n_of_dif_products    91
avg_ticket           91
dtype: int64

### Average Recency Days

In [60]:
# df2.loc[df2['customer_id'] == 17850,:]

df_aux = df2[['customer_id', 'invoice_date']].drop_duplicates().sort_values(['customer_id','invoice_date'], ascending =['False','False'])
df_aux['next_customer_id'] = df_aux['customer_id'].shift() #next cx
df_aux['previous_date'] = df_aux['invoice_date'].shift() # next  invoice dt

df_aux['avg_recency_days'] = df_aux.apply(lambda x: (x['invoice_date'] - x['previous_date']).days if x['customer_id'] == x['next_customer_id'] else np.nan, axis = 1)

df_aux = df_aux.drop(['invoice_date', 'next_customer_id', 'previous_date'], axis =1).dropna()

#avg recency
df_avg_recency_days = df_aux.groupby('customer_id').mean().reset_index()

#Merging
df_ref = pd.merge(df_ref, df_avg_recency_days, on ='customer_id', how = 'left' )

df_ref.isna().sum()
# df_aux.dtypes


customer_id             0
gross_revenue          91
recency_days           91
invoice_ammt           91
item_ammt              91
n_of_dif_products      91
avg_ticket             91
avg_recency_days     2816
dtype: int64

In [61]:
df_ref.head()

,customer_id,gross_revenue,recency_days,invoice_ammt,item_ammt,n_of_dif_products,avg_ticket,avg_recency_days
0,17850,5391.21,372.0,34.0,1733.0,297.0,18.15,35.500000
1,13047,3232.59,56.0,9.0,1390.0,171.0,18.90,27.250000
2,12583,6705.38,2.0,15.0,5028.0,232.0,28.90,23.187500
3,13748,948.25,95.0,5.0,439.0,28.0,33.87,92.666667
4,15100,876.00,333.0,3.0,80.0,3.0,292.00,8.600000


### Frequency

In [62]:
### Frequency purchase

df2_max = df2[['customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').max()
df2_min = df2[['customer_id', 'invoice_date']].drop_duplicates().groupby('customer_id').min()
# df2_purchase = df2[['customer_id','invoice_date']].drop_duplicates().groupby('customer_id').count()
df2_purchase.head()

df_aux = ( df2[['customer_id','invoice_no','invoice_date']].drop_duplicates()
                                                           .groupby('customer_id')
                                                           .agg( max_  = ('invoice_date','max'),
                                                                 min_  = ('invoice_date','min'),
                                                                 days_ = ('invoice_date',lambda x: ((x.max() - x.min() ).days) +1  ),
                                                                 buy_  = ('invoice_no', 'count' ) ) ).reset_index()

# frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply(lambda x: x['buy_'] / x['days_'] if x['days_'] != 0 else 0, axis = 1 )

# Merging
df_ref = pd.merge(df_ref, df_aux[['customer_id', 'frequency']], on = 'customer_id', how = 'left')
# df_ref.head()
df_ref.isna().sum()

# df_aux.sort_values('frequency', ascending = False).head(20)
# df_aux[df_aux['customer_id'] == 17850].head()

customer_id             0
gross_revenue          91
recency_days           91
invoice_ammt           91
item_ammt              91
n_of_dif_products      91
avg_ticket             91
avg_recency_days     2816
frequency               0
dtype: int64

In [63]:
df_ref.head()

,customer_id,gross_revenue,recency_days,invoice_ammt,item_ammt,n_of_dif_products,avg_ticket,avg_recency_days,frequency
0,17850,5391.21,372.0,34.0,1733.0,297.0,18.15,35.500000,0.486111
1,13047,3232.59,56.0,9.0,1390.0,171.0,18.90,27.250000,0.048780
2,12583,6705.38,2.0,15.0,5028.0,232.0,28.90,23.187500,0.045699
3,13748,948.25,95.0,5.0,439.0,28.0,33.87,92.666667,0.017921
4,15100,876.00,333.0,3.0,80.0,3.0,292.00,8.600000,0.136364


### Number of Returns

In [64]:
df_returns = df2_returns[['customer_id','quantity']].groupby('customer_id').sum().reset_index().rename(columns={'quantity':'qtt_returns'})
df_returns['qtt_returns'] = df_returns['qtt_returns'] * -1

df_ref = pd.merge(df_ref, df_returns, how = 'left', on ='customer_id')
df_ref.loc[df_ref['qtt_returns'].isna(), 'qtt_returns'] = 0
df_ref.isna().sum()

customer_id             0
gross_revenue          91
recency_days           91
invoice_ammt           91
item_ammt              91
n_of_dif_products      91
avg_ticket             91
avg_recency_days     2816
frequency               0
qtt_returns             0
dtype: int64

### Basket Size - Number of product items per basket  *sum

In [65]:
# invoice (basket) > products > number of total items for all products
# df2_purchase.head()

In [66]:
df_aux =(df2_purchase.loc[:,['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                      .agg(n_purchase = ('invoice_no', 'nunique'), 
                                                                           n_products = ('quantity','sum'))
                                                                      .reset_index() )


df_aux['avg_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_ref = pd.merge(df_ref, df_aux[['customer_id','avg_basket_size']], how = 'left', on = 'customer_id')
df_ref.isna().sum()
# missing NA´s the ones in 'returns'

customer_id             0
gross_revenue          91
recency_days           91
invoice_ammt           91
item_ammt              91
n_of_dif_products      91
avg_ticket             91
avg_recency_days     2816
frequency               0
qtt_returns             0
avg_basket_size        91
dtype: int64

In [67]:
df2_purchase[df2_purchase['customer_id'] == 12346]

,invoice_no,stock_code,quantity,invoice_date,unit_price,country,customer_id,gross_revenue
61619,541431,23166,74215,2017-01-16,1.04,United Kingdom,12346,77183.6


### Basket, distinctive items per purchase. *count

In [68]:
# Transaction > product > item ammount

df_aux = (df2_purchase.loc[:,['customer_id', 'invoice_no', 'quantity']].groupby('customer_id')
                                                                       .agg(n_purchase = ('invoice_no', 'nunique'), 
                                                                            n_products = ('quantity','nunique'))
                                                                       .reset_index() )

df_aux['avg_unique_basket_size'] = df_aux['n_products'] / df_aux['n_purchase']

df_ref = pd.merge(df_ref, df_aux[['customer_id','avg_unique_basket_size']], how = 'left', on = 'customer_id')
df_ref.isna().sum()

customer_id                  0
gross_revenue               91
recency_days                91
invoice_ammt                91
item_ammt                   91
n_of_dif_products           91
avg_ticket                  91
avg_recency_days          2816
frequency                    0
qtt_returns                  0
avg_basket_size             91
avg_unique_basket_size      91
dtype: int64

Check if returns will remain in df_ref:

In [69]:
df_ref.head()

,customer_id,gross_revenue,recency_days,invoice_ammt,item_ammt,n_of_dif_products,avg_ticket,avg_recency_days,frequency,qtt_returns,avg_basket_size,avg_unique_basket_size
0,17850,5391.21,372.0,34.0,1733.0,297.0,18.15,35.500000,0.486111,40.0,50.970588,0.176471
1,13047,3232.59,56.0,9.0,1390.0,171.0,18.90,27.250000,0.048780,35.0,154.444444,1.222222
2,12583,6705.38,2.0,15.0,5028.0,232.0,28.90,23.187500,0.045699,50.0,335.200000,1.600000
3,13748,948.25,95.0,5.0,439.0,28.0,33.87,92.666667,0.017921,0.0,87.800000,1.600000
4,15100,876.00,333.0,3.0,80.0,3.0,292.00,8.600000,0.136364,22.0,26.666667,0.666667


In [70]:
df_ref.shape

(5787, 12)

In [71]:
# df_returns = df2_returns[['customer_id', 'invoice_no']].drop_duplicates().groupby('customer_id').count().reset_index().rename(columns ={'invoice_no':'returns'})
# df_ref = pd.merge(df_ref, df_returns, how = 'left', on='customer_id')
# df_ref.loc[df_ref['returns'].isna(),'returns'] = 0
# df_ref.isna().sum()

# EDA

In [72]:
df4 = df_ref.dropna().copy()

df4.isna().sum()

customer_id               0
gross_revenue             0
recency_days              0
invoice_ammt              0
item_ammt                 0
n_of_dif_products         0
avg_ticket                0
avg_recency_days          0
frequency                 0
qtt_returns               0
avg_basket_size           0
avg_unique_basket_size    0
dtype: int64

## Univariate Analysis

In [73]:
# profile = ProfileReport(df4)
# profile.to_file('output.html')

### Gross Revenue

In [74]:
# df4.sort_values('gross_revenue', ascending = False).head(10)

In [75]:
# df3[df3['customer_id'] == 14646].head()

In [76]:
# np.sum(df3.loc[df3['customer_id'] == 14646,'quantity'] * df3.loc[df3['customer_id']== 14646,'unit_price'])

## Bivariate Analysis

## Space Study

In [77]:
# full_feat:
df43_full_features = df4.drop(columns = ['customer_id'], axis = 1).copy()

# embedded space:
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'n_of_dif_products', 'frequency', 'qtt_returns']
df43_less_features = df4[cols_selected].copy()
df43_less_features.drop(columns= 'customer_id', inplace = True)
# min-max, standard scales, robust scaler?

In [78]:
from sklearn import preprocessing as pp
mm = pp.MinMaxScaler()

df43_full_features['gross_revenue'] = mm.fit_transform(df43_not_embedded[['gross_revenue']])

# 'customer_id', 
df43_full_features['recency_days']          = mm.fit_transform( df43_full_features[['recency_days']])
df43_full_features['invoice_ammt']          = mm.fit_transform( df43_full_features[['invoice_ammt']])
df43_full_features['item_ammt']             = mm.fit_transform( df43_full_features[['item_ammt']])
df43_full_features['n_of_dif_products']     = mm.fit_transform( df43_full_features[['n_of_dif_products']])
df43_full_features['avg_ticket']            = mm.fit_transform( df43_full_features[['avg_ticket']])
df43_full_features['avg_recency_days']      = mm.fit_transform( df43_full_features[['avg_recency_days']])
df43_full_features['frequency']             = mm.fit_transform( df43_full_features[['frequency']])
df43_full_features['qtt_returns']           = mm.fit_transform( df43_full_features[['qtt_returns']])
df43_full_features['avg_basket_size']       = mm.fit_transform( df43_full_features[['avg_basket_size']])
df43_full_features['avg_unique_basket_size']= mm.fit_transform( df43_full_features[['avg_unique_basket_size']])



df43_less_features['gross_revenue'] = mm.fit_transform(df43_embedded[['gross_revenue']])
# 'customer_id', 
df43_less_features['recency_days']          = mm.fit_transform( df43_less_features[['recency_days']])
# df43_less_features['invoice_ammt']          = mm.fit_transform( df43_less_features[['invoice_ammt']])
# df43_less_features['item_ammt']             = mm.fit_transform( df43_less_features[['item_ammt']])
df43_less_features['n_of_dif_products']     = mm.fit_transform( df43_less_features[['n_of_dif_products']])
# df43_less_features['avg_ticket']            = mm.fit_transform( df43_less_features[['avg_ticket']])
# df43_less_features['avg_recency_days']      = mm.fit_transform( df43_less_features[['avg_recency_days']])
df43_less_features['frequency']             = mm.fit_transform( df43_less_features[['frequency']])
df43_less_features['qtt_returns']           = mm.fit_transform( df43_less_features[['qtt_returns']])
# df43_less_features['avg_basket_size']       = mm.fit_transform( df43_less_features[['avg_basket_size']])
# df43_less_features['avg_unique_basket_size']= mm.fit_transform( df43_less_features[['avg_unique_basket_size']])

df43_less_features.columns

NameError: name 'df43_not_embedded' is not defined

In [ ]:
X_less_features = df43_less_features.copy()
X_full_features = df43_full_features.copy()

In [ ]:
df43_full_features.head()

In [ ]:
df43_less_features.head()

### PCA - full features

In [ ]:
X_full_features.head()

In [ ]:
from sklearn import decomposition as dd
pca = dd.PCA( n_components = X_full_features.shape[1])
principal_component = pca.fit_transform(X_full_features)

# plot explaining variables
features = range ( pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color = 'black')

# PCA COMPONENTS
df_pca = pd.DataFrame(principal_component)

In [ ]:
df_pca.head()

In [ ]:
sns.scatterplot(x = 0, y = 1, data = df_pca)

#### PCA - Embedded Space

In [ ]:
X_less_features.head()

In [ ]:
pca = dd.PCA( n_components = X_less_features.shape[1])
principal_components = pca.fit_transform(X_less_features)

# plot explaining variables
features = range ( pca.n_components_)
plt.bar(features, pca.explained_variance_ratio_, color = 'black')

# PCA COMPONENTS
df_pca = pd.DataFrame(principal_components)

In [ ]:
sns.scatterplot( x=0, y=1, data=df_pca )

In [ ]:
# no clear spacing

### UMAP - full features

In [ ]:
# UMAP
import umap.umap_ as umap
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(X_not_emb)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )

#### UMAP - Less features

In [ ]:
# UMAP
import umap.umap_ as umap
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(X_less_features)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )

In [ ]:
# still not many grouping found

### t-SNE - full features

In [ ]:
# UMAP
import umap.umap_ as umap
from sklearn.manifold import TSNE
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = TSNE(n_components = 2, random_state = 42, n_jobs = -1)
embedding = reducer.fit_transform(X_full_features)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )

### t-SNE - less features

In [ ]:
# UMAP
import umap.umap_ as umap
from sklearn.manifold import TSNE
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = TSNE(n_components = 2, random_state = 42, n_jobs = -1)
embedding = reducer.fit_transform(X_less_features)

# embedding
df_pca['embedding_x'] = embedding[:,0]
df_pca['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df_pca )

### Tree-Based Embedding - v1.0

In [ ]:
from sklearn import ensemble as en

In [ ]:
df4.head()
X = df4.drop( columns = ['customer_id', 'gross_revenue'], axis = 1 )
y = df4['gross_revenue'] # using gross_revenue

# model definition
rf_model = en.RandomForestRegressor( random_state = 42, n_estimators = 100)

# model training
rf_model.fit( X, y)

#leaf
# rf_model.apply( X )  # applying own training data

#dataframe Leag

In [ ]:
df_leaf = pd.DataFrame(rf_model. apply(X))
df_leaf.head()

In [ ]:
# each point in the table is the position the customer was assigned to
X.shape

In [ ]:
df_leaf.shape  # 100 (the 100 estimators)

In [ ]:
# Reducing dimensionality  # reduce the projection of the 100 columns into 2.
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf)

#embedding
df_tree = pd.DataFrame()
df_tree['embedding_x'] = embedding[:,0]
df_tree['embedding_y'] = embedding[:,1]

#plotting
sns.scatterplot(x = 'embedding_x',
               y = 'embedding_y',
               data = df_tree)


#### Tree-Based Embedding - v2.0

In [ ]:
df4.head()
X = df4.drop( columns = ['customer_id', 'gross_revenue'], axis = 1 )
y_aux = pd.DataFrame()
y_aux['gross_revenue'] = df4['gross_revenue']
y_aux['gross_revenue'] = mm.fit_transform(y_aux[['gross_revenue']])
y = y_aux['gross_revenue'] # using gross_revenue


for a in X.columns:
    X[a] = mm.fit_transform( X[[a]])


# model definition
rf_model = en.RandomForestRegressor( random_state = 42, n_estimators = 100)

# model training
rf_model.fit( X, y)

#leaf
# rf_model.apply( X )  # applying own training data

#dataframe Leag

df_leaf = pd.DataFrame(rf_model. apply(X))
df_leaf.head()

In [ ]:
y.head()

In [ ]:
# Reducing dimensionality  # reduce the projection of the 100 columns into 2.
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf)

#embedding
df_tree_v2 = pd.DataFrame()
df_tree_v2['embedding_x'] = embedding[:,0]
df_tree_v2['embedding_y'] = embedding[:,1]

#plotting
sns.scatterplot(x = 'embedding_x',
               y = 'embedding_y',
               data = df_tree_v2)

In [ ]:
# Now we run the new space on the data.so people become neighbors because of the gross_revenue.  
# 'so they are neighbors in the embedded space'.
# This works, but wont be able to explain the reason neighbors are neighbors....
# 

# Data Preparation


In [ ]:
# df5 = df4.copy()
df5_aux = df4.copy()
df5 = df_tree.copy()

In [ ]:

sns.distplot(df5_aux['gross_revenue']);

In [ ]:
# Talvez um minmax, tá muito deseguilibrada essa disribuicao.


In [ ]:
# sns.distplot(np.log(df5['gross_revenue']))

In [ ]:
df5_aux.columns

In [ ]:
# mm = pp.MinMaxScaler()
# ss = pp.StandardScaler()
# rs = pp.RobustScaler()

df5_aux['gross_revenue'] = mm.fit_transform(df5_aux[['gross_revenue']])
df5_aux['recency_days'] = mm.fit_transform(df5_aux[['recency_days']])
df5_aux['invoice_ammt'] = mm.fit_transform(df5_aux[['invoice_ammt']])
df5_aux['item_ammt'] = mm.fit_transform(df5_aux[['item_ammt']])
df5_aux['n_of_dif_products'] = mm.fit_transform(df5_aux[['n_of_dif_products']])

# df5_aux['avg_ticket'] = mm.fit_transform(df5_aux[['avg_ticket']])
# df5_aux['avg_recency_days'] = mm.fit_transform(df5_aux[['avg_recency_days']])

df5_aux['frequency'] = mm.fit_transform(df5_aux[['frequency']])
df5_aux['qtt_returns'] = mm.fit_transform(df5_aux[['qtt_returns']])

# df5_aux['avg_basket_size'] = mm.fit_transform(df5_aux[['avg_basket_size']])
# df5_aux['avg_unique_basket_size'] = mm.fit_transform(df5_aux[['avg_unique_basket_size']])  

# avg wont make sense in the end, not looking for avg of the avg in the end.

variable = 'gross_revenue'

# ['customer_id', 'gross_revenue', 'recency_days', 'n_of_dif_products', 'frequency', 'qtt_returns']

In [ ]:
# Data as it is
print('Min:{} - Max:{}'.format(df5_aux[variable].min(), df5_aux[variable].max() ) )
sns.displot(df5_aux[variable]);

In [ ]:
# # Data Standardized/Rescaled
print('Min:{} - Max:{}'.format(df5_aux[variable].min(), df5_aux[variable].max() ) )
sns.displot(df5_aux[variable]);

In [ ]:
# Box Plot
sns.boxplot(df5_aux[variable]);

# Feature Selection

In [ ]:
# cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'n_of_dif_products', 'frequency', 'qtt_returns']

In [ ]:
# df6 = df5[cols_selected].copy()
df6 = df_tree.copy()
df6.head()

# Hyperparameter Fine-Tuning

In [ ]:
# X = df6.drop(columns = ['customer_id'])
X = df_tree.copy()
X_v2 = df_tree_v2.copy()

In [ ]:
X.head()

In [ ]:
# clusters = [2, 3, 4, 5, 6, 7 , 8 ,9 ]
clusters = np.arange(2,25,1)

## K-Means

In [ ]:
from sklearn import metrics as m
kmeans_list = []
for k in clusters:
    # defining model
    kmeans_model = c.KMeans(n_clusters = k)

    # defining training
    kmeans_model.fit(X)

    # define prediction
    labels = kmeans_model.predict(X)
    # performance
    # Using silhouette inside metrics package.
    kmeans_list.append( m.silhouette_score(X, labels, metric = 'euclidean') )
    

#### v2.0 

kmeans_list_v2 = []
for k in clusters:
    # defining model
    kmeans_model = c.KMeans(n_clusters = k)

    # defining training
    kmeans_model.fit(X_v2)

    # define prediction
    labels = kmeans_model.predict(X_v2)
    # performance
    # Using silhouette inside metrics package.
    kmeans_list_v2.append( m.silhouette_score(X_v2, labels, metric = 'euclidean') )

In [ ]:
plt.plot(clusters, kmeans_list, linestyle='--', marker = 'o', color ='b')
plt.plot(clusters, kmeans_list_v2, linestyle='--', marker = 'o', color ='green')
plt.xlabel('K')
plt.ylabel(' Silhouette Score ')
plt.title('KMeans - Silhouette Score x K')

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

blue_patch = mpatches.Patch(color='blue', label='Raw data')
green_patch = mpatches.Patch(color='green', label='MinMax Transformation')

plt.legend(handles=[blue_patch, green_patch])



In [ ]:
np.unique(labels)

## GMM

In [ ]:
from sklearn import mixture as mx

In [ ]:
# def, training, predict, score
gmm_list =[]
for k in clusters:
    gmm_model = mx.GaussianMixture(n_components = k)

    gmm_model.fit(X)

    labels = gmm_model.predict(X)

    sil = m.silhouette_score(X, labels, metric = 'euclidean')
    gmm_list.append(sil)

    
### v2.0
gmm_list_v2 =[]
for k in clusters:
    gmm_model = mx.GaussianMixture(n_components = k)

    gmm_model.fit(X_v2)

    labels = gmm_model.predict(X_v2)

    sil = m.silhouette_score(X_v2, labels, metric = 'euclidean')
    gmm_list_v2.append(sil)
    

In [ ]:
plt.plot(clusters, gmm_list, linestyle='--', marker = 'o', color = 'b')
plt.plot(clusters, gmm_list_v2, linestyle='--', marker = 'o', color = 'green')
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.title('GMM Silhouette Score K')

## Hierarchical Clustering

In [ ]:
from scipy.cluster import hierarchy as hc

In [ ]:
# model definition and training
hc_model = hc.linkage(X, 'ward')
hc_model_v2 = hc.linkage(X_v2, 'ward')



In [ ]:
# DENDROGRAM

# hc.dendrogram(
#     hc_model,
#     leaf_rotation = 90,
#     leaf_font_size = 8)
# plt.plot()

In [ ]:
hc.dendrogram(
    hc_model,
    truncate_mode='lastp',
    p=12,
    leaf_rotation = 90,
    leaf_font_size = 8,
    show_contracted = True)
plt.show()

### H-Clustering Silhouette Score

In [ ]:
hc_list = []
for k in clusters:
    # def, training, predict score
    hc_model = hc.linkage(X, 'ward')

    # prediction
    labels = hc.fcluster(hc_model, k, criterion = 'maxclust')

    #metrics
    sil = m.silhouette_score(X, labels, metric = 'euclidean')
    hc_list.append( sil )

    
#### v2.0
hc_list_v2 = []
for k in clusters:
    # def, training, predict score
    hc_model_v2 = hc.linkage(X_v2, 'ward')

    # prediction
    labels = hc.fcluster(hc_model_v2, k, criterion = 'maxclust')

    #metrics
    sil = m.silhouette_score(X_v2, labels, metric = 'euclidean')
    hc_list_v2.append( sil )

In [ ]:
plt.plot(clusters, hc_list, linestyle = '--', marker ='o', color = 'b')
plt.plot(clusters, hc_list_v2, linestyle = '--', marker ='o', color = 'green')
plt.xlabel('k')
plt.ylabel('')

In [ ]:
## DBSCAN

In [ ]:
## Results

In [ ]:
# Model Name / k =2 / k=3 ....
# K Means     SS      SS
# 

# Yellowbrick nao tem os dados da silhoutte na funcao, tem que fazer na mao pra ver a silhouette
# em R já é tudo pronto. em python nao.
# No R é mais fácil fazer clustering


In [ ]:
# K means encontra clusters redondos - usa a distância min entre pontos.
# Nem todos os clusters formam um círculo, alguns formam 'tripas' etc
# Usar clusters baseados em distribuição de probabilidade.
# Assumir que os clusters sao distribuicoes de probabilidades.
#algoritmo EM Expectation-Maximize  -> Calculates expected value and maximization


## DBSCAN

In [ ]:
eps = 1.2
min_samples = 33
dbscan_model = c.DBSCAN( eps = eps, min_samples = min_samples) 

labels = dbscan_model.fit_predict(X)

db_scan_sil = m.silhouette_score( X, labels, metric = 'euclidean')
sil
print('Silhouette Score {} '.format(db_scan_sil))
print('Clusters inc noisy one (-1) if present: {}'.format(len(unique(labels) )))
print('labels {} '.format(unique(labels)))

print('\n\nVersion2:')
dbscan_model_v2 = c.DBSCAN( eps = eps, min_samples = min_samples) 

labels_v2 = dbscan_model_v2.fit_predict(X_v2)

db_scan_sil_v2 = m.silhouette_score( X_v2, labels, metric = 'euclidean')
sil
print('Silhouette Score {} '.format(db_scan_sil_v2))
print('Clusters inc noisy one (-1) if present: {}'.format(len(unique(labels_v2) )))
print('labels {} '.format(unique(labels)))

In [ ]:
db_scan_k = len(unique(labels)) - 1


In [ ]:
dbscan_list = []

    

### Neighbors issue

In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:
neighbors = NearestNeighbors(n_neighbors = min_samples).fit(X)
distances, indices = neighbors.kneighbors(X)

In [ ]:
distances = np.sort(distances, axis = 0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
plt.plot(distances[2900:])

In [ ]:
labels

## RESULTS

In [ ]:
df_results = pd.DataFrame(
    {'KMeans':kmeans_list, 'GMM': gmm_list,
     'HCList':hc_list}).T
# todo - include dbscan

dbscan_list = []
for a in (df_results.columns):
    if a != db_scan_k:
        dbscan_list.append(0.0)
    else:
        dbscan_list.append(db_scan_sil)

print(dbscan_list)
    
df_results = pd.DataFrame(
    {'KMeans':kmeans_list, 'GMM': gmm_list,
     'KMeans_v2':kmeans_list_v2, 'GMM_v2': gmm_list_v2,
     'HCList_v2':hc_list_v2,'HCList':hc_list, 'DBScan':dbscan_list}).T

df_results.columns = clusters

df_results.style.highlight_max(color = 'lightgreen', axis = 1)

## Script Silhouette

In [ ]:
# k = 2

fig, ax = plt.subplots(12,2)
fig.set_size_inches(25,20)  #setting a size for fig

for k in clusters:
    q,mod = divmod(k,2)
    ax[q-1, mod].set_xlim([ -0.1, 1])
    ax[q-1, mod].set_ylim([ 0, len(X) + (k+1)*10])
    #model def
    hc_model = hc.linkage(X, 'ward')

    # predict
    labels = hc.fcluster(hc_model, k, criterion = 'maxclust')

    # performance
    ss = m.silhouette_score(X, labels, metric = 'euclidean')
    print('For k {}, Silhouette Score: {}'.format(k, ss))
    samples_silhouette_values = m.silhouette_samples(X, labels)
    y_lower = 10
    for i in range(k):
        # select clusters
        ith_samples_silhouette_values = samples_silhouette_values[labels == i]

        #sorting
        ith_samples_silhouette_values.sort()
        
        # size_cluster
        size_cluster_i = ith_samples_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i
        
        cmap = cm.get_cmap('Spectral')
        color = cmap(i/k)
                


        # fill_betweenx  (matplotlib)

        

        ax[q-1,mod].fill_betweenx( np.arange(y_lower, y_upper), 0, ith_samples_silhouette_values)
        y_lower = y_upper+ 20
    
    ax[q-1, mod].set_yticks([])
    ax[q-1, mod].set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])


    # silhouette analysis sklearn - check sklearn suggestions for scripts.
    # plot kmeans silhouette analysis -- check their suggestions
    

## Within-Cluster Sum of Square (WSS)

In [ ]:
# trying out diffente cluster values

clusters = [2, 3, 4, 5, 6, 7 ]

In [ ]:
wss = []
for k in clusters:
    # model definition
    kmeans = c.KMeans( init = 'random', n_clusters = k, n_init = 10, max_iter = 300, random_state = 42)  # doing centroids at random # n_init, centroid starting times(?)
    # model training
    kmeans.fit(X)
    # validation
    wss.append(kmeans.inertia_)

# plootting wss - ELBOW method # We want to findout the best k before going for the final model
# like a pre-training model.
# This graph is supposed to look like an elbow... (?)
# Supposed to choose the point in the graph where the changes(The angle) most significative happen in this case
# happens where K = 3 and 5 - it is up to the Data Scientist to choose the best point.b

plt.plot(clusters, wss, linestyle = '--', marker ='o', color='b')
plt.xlabel('K')
plt.ylabel('Within-Cluster Sum of Square');
plt.title('WSS vs K')

In [ ]:
# Choosing K using a more elegant way, using the library yellowbrick
from yellowbrick.cluster import KElbowVisualizer
# !pip install yellowbrick.cluster

kmeans = KElbowVisualizer(c.KMeans(), k=clusters, timing = False)
kmeans.fit(X)
kmeans.show()  # it draws the method and tells you the best k value...

# There is also the knee method....

In [ ]:
wss

In [ ]:
# In grouping / clustering WSS -> 

## Sillhouette Score

In [ ]:
kmeans_si = KElbowVisualizer(c.KMeans(), k=clusters, metric = 'silhouette', timings = False)
kmeans_si.fit(X)
kmeans_si.show()

In [ ]:
X.head()

In [ ]:
# elbow deu '3', silhouette deu 2, qual escolher?
# a silhouetta, é melhor...
# cotovelo nao considera o tamanho do cluster.


## Silhouette Analysis

In [ ]:
fig, ax = plt.subplots(3,2, figsize = (25,18))
for k in clusters:
    km = c.KMeans(n_clusters = k, init = 'random', n_init = 10, max_iter = 100, random_state = 42)
    q, mod = divmod(k, 2)
    # visualizer = SilhouetteVisualizer(km, colors = 'yellowbrick') #, ax=[0][0])
    visualizer = SilhouetteVisualizer(km, colors = 'yellowbrick', ax=ax[q-1][mod])
    visualizer.fit(X)
    visualizer.finalize()

Silhouette now giving us much insight on the number of clusters, sticking with elbow so far - more features needed.

# Model Training

## K-Means

In [ ]:
# model definition
k = 11
kmeans = c.KMeans(init = 'random', n_clusters = k, n_init = 10, max_iter = 300, random_state = 42)

# model training
kmeans.fit(X)

# clustering
# labels = kmeans.predict(X)
labels = kmeans.labels_


## Cluster Validation

In [ ]:
from sklearn import metrics as m

In [ ]:
# WSS (Whithin Cluster Sum of Squares)
print( f'WSS Value: {kmeans.inertia_}')

# SS ( Silhouette Score)
print(f'SS Value: {m.silhouette_score(X, labels, metric = "euclidean")}')

# Cluster Analysis

## Visual Inspection

In [ ]:
# df9 = df6.copy()
df9 = X.copy()
df9['cluster'] = labels
df9.head()

In [ ]:
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', data = df9, hue = 'cluster', palette = 'deep')

### Multi-Feature

In [ ]:
visualizer = SilhouetteVisualizer(kmeans, colors = 'yellowbrick')
visualizer.fit(X)
visualizer.finalize()

### 2d plot

In [ ]:
df_viz = df9.drop(columns = 'customer_id', axis = 1)
# sns.pairplot(df_viz, hue = 'cluster')

### UMAP - t-SNE

In [ ]:
# Manifold
# UMAP, t-SNE(2009) - MAnifold - Topology


In [ ]:
# !pip install umap-learn

In [ ]:
# UMAP
import umap.umap_ as umap
# reducer = umap.UMAP(n_neighbors = 9, random_state = 42)
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(X)

# embedding
df_viz['embedding_x'] = embedding[:,0]
df_viz['embedding_y'] = embedding[:,1]

#plotting umap
sns.scatterplot( x = 'embedding_x', y = 'embedding_y', 
                hue = 'cluster', 
                palette = sns.color_palette('hls', n_colors = len( df_viz['cluster'].unique() ) ), data = df_viz )

### 3 features 3d.

In [ ]:


# fig = px.scatter_3d( df9, x='recency_days', y='invoice_no', z = 'gross_revenue', color = 'cluster')
# fig.show()

## Cluster Profile


In [ ]:
df9.head()

In [ ]:

df9_aux = df4.copy()
df9_aux['cluster'] = labels

# Number of Customers
df_cluster = (df9_aux[['customer_id', 'cluster']].groupby('cluster').count().reset_index())
df_cluster['perc_customer'] = 100 * (df_cluster['customer_id'] / df_cluster['customer_id'].sum())


# Centroids - the group mean. 

# Avg Gross Revenue
df_avg_gross_revenue = (df9_aux[['gross_revenue', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_avg_gross_revenue, on = 'cluster', how = 'left')

# Avg recency days
df_avg_recency_days = (df9_aux[['recency_days', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_avg_recency_days, on = 'cluster', how = 'left')

# Avg invoices number
df_inv = (df9_aux[['n_of_dif_products', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_inv, on = 'cluster', how = 'left')

# Frequency
df_fre = (df9_aux[['frequency', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_fre, on = 'cluster', how = 'left')

# Returns
df_ret = (df9_aux[['qtt_returns', 'cluster']].groupby('cluster').mean().reset_index())
df_cluster = pd.merge(df_cluster, df_ret, on = 'cluster', how = 'left')

df_cluster


In [ ]:
X -> Insiders,
X -> Needs to improve frequency
X -> ....

## Hypothesis Testing

### MindMap

- Modeling the phenomena.
- Understanding entities (customer, location, family, income)
- Entity properties (eg. customer: income, education, age group, etc)


In [ ]:
from IPython.core.display import Image
Image('./mindmap.png')

### Hypothesis Purchases: 

1 - Insiders cluster customers use credit card for more than 80% of purchases.

2 - Customers in the Insiders cluster have an average ticket 10% above the More Products cluster.

3 - Insiders cluster customers have a basket size of over 5 products.

4 - Insiders cluster customers have a volume (products) of purchases above 10% of total purchases.

5 - Insiders cluster customers have a volume (turnover) of purchases above 10% of total purchases.

6 - Customers in the Insiders cluster have a number of returns below the average of the total customer base.

7 - The median billing by insider cluster customers is 10% higher than the median overall billing.

8 - The gmv percentile by insider customers.



Cluster 1: (Insider Candidates - 0.14%)
    - Number of Customers: 6
    - Avg Recency: 7 days
    - Avg purchases: 89
    - Avg Revenue: 182.182,00

Cluster 2: (0.71 % Customers)
    - Number of Customers: 31
    - Avg Recency: 13 days
    - Avg purchases: 53
    - Avg Revenue: 40.543,52    
    
Cluster 3: (99% Customers)
    - Number of Customers: 4.335
    - Avg Recency: 92 days
    - Avg purchases: 5
    - Avg Revenue: 1.372,57

# Deploy to production